In [1]:
!pip install galois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 9.7 MB/s eta 0:00:00


## galois library documentation :

https://pypi.org/project/galois/

https://mhostetter.github.io/galois/latest/basic-usage/array-arithmetic/

In [2]:
import math
import numpy as np
import random
import galois

# **Reed-Solomon Codes**


## Reed-Solomon Encoder


In [186]:
# q must be prime and bigger than n
# n >>> code length
# q >>> field of elements
# k >>> length of input message
# e >>> maximum length of error possible

q = 31
n = 31
k = 14

const_needed = 1

e = math.floor((n-k)/2)

print(const_needed)
print(e)

1
8


### Defining field

In [154]:
GF_q = galois.GF(q)

y = np.array([5]); y

print(y.view(GF_q))

[5]


### creating random input message

In [185]:
# Assigning seed to make reproducible results
np.random.seed(1)

# creating random input message
message1 =  np.random.randint(0,q,k)

message2 = message1.view(GF_q)

message = np.flip(message2)

print('\n\nGenerated Random input message in GF(q): \n')
print(message2)

# message polynomial
Message_poly = galois.Poly(message, GF_q)


print('\n\ninput message polynomial: \n')
print(Message_poly)



Generated Random input message in GF(q): 

[ 5 11 12  8  9 11  5 15  0 16  1 12  7 13]


input message polynomial: 

13x^13 + 7x^12 + 12x^11 + x^10 + 16x^9 + 15x^7 + 5x^6 + 11x^5 + 9x^4 + 8x^3 + 12x^2 + 11x + 5


### Choosing n distinct Alpha

In [176]:
random.seed(42)

alpha1 = random.sample(range(0,q), n)

alpha2 = np.array(alpha1)

alpha = alpha2.view(GF_q); alpha2

print('\n\n printing Alphas: \n')
print((alpha))




 printing Alphas: 

[20  3  0 23  8  7 25  4 29 21 17  2 18 13  1 28 16 22 30 26  9 15 11 12
  5  6 27 24 10 14 19]


## Encoding input message

In [177]:
Code = Message_poly(alpha)

print('\n\n encoded message (X): \n')
print(Code)



 encoded message (X): 

[ 8 28  5 22  1 13 15 16  5 29  8  2 29  4  1 25  3  5 15  7  9 13 25  3
 14 19 13  3  1 30  1]


# **Adding Error**

In [178]:
Code = Message_poly(alpha)

W = Code


print('\n\n encoded message (X): \n')
print(W)

random.seed(46)


########## Specify number of errors (err) ###########

err = 4

########## Specify number of errors (err) ###########

index = random.sample(range(0,n), err)

add_err1 = random.sample(range(1,q), err)

add_err = np.array(add_err1)

if err :

  add_err = add_err.view(GF_q)

  for i in range(0,err):
    W[index[i]] = add_err[i] + W[index[i]]


print('\n\n index of message to add error with: \n')
print(index)

print('\n\n value of error to be added: \n')
print(add_err)

print('\n\n Received message (X + E): \n')
print(W)





 encoded message (X): 

[ 8 28  5 22  1 13 15 16  5 29  8  2 29  4  1 25  3  5 15  7  9 13 25  3
 14 19 13  3  1 30  1]


 index of message to add error with: 

[28, 2, 12, 1]


 value of error to be added: 

[19 28 29  8]


 Received message (X + E): 

[ 8  5  2 22  1 13 15 16  5 29  8  2 27  4  1 25  3  5 15  7  9 13 25  3
 14 19 13  3 20 30  1]


# **Reed-Solomon Decoder**

## System Function Generator
### Q(alpha) = Wi*E(alpha)

In [179]:
def sys_mat(W,alpha,er,const_needed,n,k):
  Q_matrix = np.full((n,k+er),0)
  E_matrix = np.full((n,er+1),0)
  for i in range(0,n):
    for j in range(0,k+er):
      Q_matrix[i,j] = alpha[i] ** j

    for j in range(0,1+er):
      E_matrix[i,j] = -(W[i]*(alpha[i] ** j))
      if (j == er) & (const_needed == 1):
        E_matrix[i,j] = (W[i]*(alpha[i] ** j))
  A = np.concatenate((Q_matrix,E_matrix), axis = 1)
  A = A.view(GF_q)
  return A

### Finding an E degree, for which the system is solvable

In [180]:
solvable = []

for i in range(0,e+1):
  H = sys_mat(W,alpha,i,const_needed,n,k)
  Rank = np.linalg.matrix_rank(H)
  sh = H.shape
  Col_num = sh[-1]
  print(Rank , Rank - (Col_num - const_needed))
  if Rank - (Col_num - const_needed) == 0:
    solvable.append(i)

print (solvable)


15 1
17 1
19 1
21 1
22 0
23 -1
24 -2
25 -3
26 -4
[4]


## Solving system of equations

In [181]:
A1 = sys_mat(W,alpha,solvable[0],const_needed,n,k)

print('\n\n joint raw system matrix ([A b]) as in AX=b: \n')
print (A1)

space = A1.row_space()

print('\n\n joint system matrix ([A b]) as in AX=b after gaussian elimination: \n')
print(space)

b = space[:,-1]
b = b.view(GF_q)

print('\n\n b vector as in Ax=b: \n')
print(b)

A1_F = np.delete(space ,-1 ,axis = 1)


print('\n\n Final system matrix after gaussian elimination: \n')
print(A1_F)


# sloving the system of equations in galois field q
# https://mhostetter.github.io/galois/latest/basic-usage/array-arithmetic/#linear-algebra

Q_E = np.linalg.solve(A1_F, b)



print('\n\n printing joint Q and E coefficients: \n')

print(Q_E)



 joint raw system matrix ([A b]) as in AX=b: 

[[ 1 20 28  2  9 25  4 18 19  8  5  7 16 10 14  1 20 28 23 26 24 15 10]
 [ 1  3  9 27 19 26 16 17 20 29 25 13  8 24 10 30 28 22 26 16 17 20  2]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 29  0  0  0  0]
 [ 1 23  2 15  4 30  8 29 16 27  1 23  2 15  4 30  8 29  9 21 18 11 26]
 [ 1  8  2 16  4  1  8  2 16  4  1  8  2 16  4  1  8  2 30 23 29 15  4]
 [ 1  7 18  2 14  5  4 28 10  8 25 20 16 19  9  1  7 18 18  2 14  5 27]
 [ 1 25  5  1 25  5  1 25  5  1 25  5  1 25  5  1 25  5 16 28 18 16  3]
 [ 1  4 16  2  8  1  4 16  2  8  1  4 16  2  8  1  4 16 15 29 23 30  4]
 [ 1 29  4 23 16 30  2 27  8 15  1 29  4 23 16 30  2 27 26 10 11  9 18]
 [ 1 21  7 23 18  6  2 11 14 15  5 12  4 22 28 30 10 24  2 11 14 15 26]
 [ 1 17 10 15  7 26  8 12 18 27 25 22  2  3 20 30 14 21 23 19 13  4 25]
 [ 1  2  4  8 16  1  2  4  8 16  1  2  4  8 16  1  2  4 29 27 23 15  1]
 [ 1 18 14  4 10 25 16  9  7  2  5 28  8 20 19  1 18 14  4 10 25 16 22]
 [ 1 13 14 27 1

### Finding E and Q

In [182]:
temp = Q_E.tolist()

E_coefs = [1]

for i in range(0,solvable[0]):
  E_coefs.append(temp[-1])
  temp.pop()

temp = np.array(temp)

E_coefs = np.array(E_coefs)
E_coefs = E_coefs.view(GF_q)


print("\n\ncoefficients of E(x) polynomial : \n")
print(E_coefs)

Q_coefs = temp

Q_coefs = np.array(Q_coefs)
Q_coefs = np.flip(Q_coefs)
Q_coefs = Q_coefs.view(GF_q)


print("\n\ncoefficients of Q(x) polynomial : \n")
print(Q_coefs)



coefficients of E(x) polynomial : 

[ 1  0 16 18  0]


coefficients of Q(x) polynomial : 

[13  7  3  6 24 15 10 14  3 18 26 13 22  0 20 30 28  0]


## Dividing Q by E and finding P(x)

In [183]:
E_poly = galois.Poly(E_coefs , field = GF_q)

Q_poly = galois.Poly(Q_coefs , field = GF_q)

P_poly = Q_poly // E_poly


print("message polynomial : \n")
print (P_poly)

Decoded_message = P_poly.coeffs

Decoded_message = np.flip(Decoded_message)

print("\n\ninput message was: \n")
print (Decoded_message)

message polynomial : 

13x^13 + 7x^12 + 12x^11 + x^10 + 16x^9 + 15x^7 + 5x^6 + 11x^5 + 9x^4 + 8x^3 + 12x^2 + 11x + 5


input message was: 

[ 5 11 12  8  9 11  5 15  0 16  1 12  7 13]
